# Collect demonstrations

Interactive notebook for collecting demonstrations

In [1]:
%matplotlib auto

import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
import json
import random
import socket
from datetime import datetime
from demonstration_recorder import DemonstrationRecorder

# the gamepad is only available on some machines, and only on Linux
available_gamepad = True
try:
    from robotcontrol.gamepad_controller import GamepadController
except ModuleNotFoundError:
    print("Approxeng module not found, cannot use gamepad")
    available_gamepad = False
    
from robotcontrol.keyboard_controller import KeyboardController
from robotcontrol.program_controller import ProgramController

# the robot is not always available, this allows us to run the demonstration 
# without the robot
available_robot = True
if available_robot:
    from robot.al5d_position_controller import PositionController, RobotPosition

from camera.camera_controller import CameraController
from demonstration import Demonstration


Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
experiment = "demonstration_collector"
# picking up a collection specific name
if socket.gethostname() == "raven":
    run = "collect_freeform_raven"
else:
    run = "collect_freeform"

exp = Config().get_experiment(experiment, run)
exp_target = Config().get_experiment("demonstration", exp["demonstration_name"])
exp_position_controller = Config().get_experiment(
    exp["exp_position_controller"], 
    exp["run_position_controller"])

exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])
exp_keyboard_controller = Config().get_experiment(exp["exp_keyboard_controller"], exp["run_keyboard_controller"])
exp_gamepad_controller = Config().get_experiment(exp["exp_gamepad_controller"], exp["run_gamepad_controller"])

***ExpRun**: Loading pointer config file:
	/home/al5d/.config/BerryPicker/mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	~/WORK/BerryPicker/cfg/settings.yaml
***ExpRun**: Configuration for exp/run: demonstration_collector/collect_freeform_raven successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/demonstration/_defaults_demonstration.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: demonstration/freeform-raven successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/position_controller_raven successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/controllers/_defaults_controllers.yaml was empty, ok.
***ExpRun**: Configuration for exp/run: controllers/camera_controller_raven successfully loaded
***ExpRun**: Experiment default config /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/

In [3]:
# starting the robot controller
if exp["available_robot"]:
    robot_controller = PositionController(exp_position_controller)
else:
    robot_controller = None
# starting the camere controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True


task_dir = exp_target.data_dir()
demoname = datetime.now().strftime('%Y_%m_%d__%H_%M_%S')
demo_dir = pathlib.Path(task_dir, demoname)
demo_dir.mkdir(parents=False, exist_ok=False)
print(f"Ok, demonstration with go into\n\t {demo_dir}")

demonstration = Demonstration(exp_target, demoname, parse_old_style = False)
demonstration.metadata["cameras"] = sorted(camera_controller.capture_devs.keys())
demonstration.save_metadata()
# fixme need to add the cameras into the demonstration

***ExpRun**: Configuration for exp/run: robot_al5d/pulse_controller_00 successfully loaded
***ExpRun**: Configuration for exp/run: robot_al5d/angle_controller_00 successfully loaded
Try out the backup /dev/ttyUSB0
cap2 works
cap3 works
Ok, demonstration with go into
	 /home/al5d/WORK/BerryPicker/data/demonstration/freeform-raven/2025_07_19__12_38_17


In [4]:
# Performs the recording. The robot is controlled through the primary 
# controller. This is also used to terminate the collection
# The information is collected by the demonstration recorder

controller = exp["primary_controller"]

if controller == "xbox":
    gamepad_controller = GamepadController(exp_gamepad_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration,
        camera_controller=camera_controller, remote_control=gamepad_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    gamepad_controller.demonstration_recorder = demo_recorder
    gamepad_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")
if controller == "keyboard":
    kb_controller = KeyboardController(exp_keyboard_controller,
        robot_controller=robot_controller, camera_controller=camera_controller)
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=kb_controller, robot_controller=
                                robot_controller, save_dir=demo_dir)
    # dr = None
    kb_controller.demonstration_recorder = demo_recorder
    kb_controller.control()
    demonstration.save_metadata()
    print("====== Demonstration terminated and recorded successfully, bye. ======")

if controller == "program":
    program_controller = ProgramController(
        robot_controller=robot_controller, camera_controller=camera_controller)
    # create wpcount 
    waypoints = []
    wpcount = 10
    while True:
        norm = [0] * 6
        for df in range(6):
            norm[df] = random.random()                
        rp = RobotPosition.from_normalized_vector(norm)
        if RobotPosition.limit(rp):
            print(f"added waypoint {rp}")
            waypoints.append(rp)
        if len(waypoints) >= wpcount:
            break

    program_controller.waypoints = waypoints
    program_controller.interactive_confirm = False
    demo_recorder = DemonstrationRecorder(demonstration, camera_controller=camera_controller, remote_control=program_controller, robot_controller=
                                robot_controller, save_dir=demo_dir, task_name=task_dir.name)
    program_controller.demonstration_recorder = demo_recorder
    demonstration.save_metadata()
    program_controller.control()


INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:5.00 distance:5.00 heading:0.00 wrist_angle:-45.00 wrist_rotation:75.00 gripper:100.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started


Found a joystick and connected
{'axes': ['l', 'lt', 'lx', 'ly', 'r', 'rt', 'rx', 'ry'], 'buttons': ['circle', 'cross', 'ddown', 'dleft', 'dright', 'dup', 'home', 'l1', 'l2', 'r1', 'r2', 'select', 'square', 'start', 'triangle']}


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/al5d/WORK/BerryPicker/vm/berrypickervenv/lib/python3.10/site-packages/cv2/qt/plugins"
_IceTransSocketUNIXConnect: Cannot connect to non-local host lboloni-Lenovo-Y50-70
_IceTransSocketUNIXConnect: Cannot connect to non-local host lboloni-Lenovo-Y50-70
Qt: Session management error: Could not open network socket
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Position:  height:5.00 distance:5.00 heading:0.00 wrist_angle:-45.00 wrist_rotation:75.00 gripper:100.00
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot done.
INFO:robotcontrol.abstract_controller:***AbstractController***: Update started
INFO:robotcontrol.abstract_controller:***AbstractController***: Update done
INFO:robotcontrol.abstract_controller:***AbstractController***: Control robot: move to position Pos

['square']
Done
====== Demonstration terminated and recorded successfully, bye. ======


In [ ]:
demonstration.save_metadata()
demonstration.move_to_video(delete_img_files=True)


***Demonstration***: moving to video started
move to video per camera dev2
move to video per camera dev3
***Demonstration***: moving to video done


In [6]:
print(demonstration.metadata)

{'stored_as_video': True, 'stored_as_images': True, 'maxsteps': 80, 'cameras': ['dev2', 'dev3']}


In [7]:
exp

Experiment:
    available_robot: true
    data_dir: /home/al5d/WORK/BerryPicker/data/demonstration_collector/collect_freeform_raven
    demonstration_name: freeform-raven
    exp_camera_controller: controllers
    exp_gamepad_controller: controllers
    exp_keyboard_controller: controllers
    exp_position_controller: robot_al5d
    exp_run_sys_indep_file: /home/al5d/WORK/BerryPicker/src/BerryPicker/src/experiment_configs/demonstration_collector/collect_freeform_raven.yaml
    experiment_name: demonstration_collector
    primary_controller: xbox
    run_camera_controller: camera_controller_raven
    run_gamepad_controller: gamepad_x360_controller
    run_keyboard_controller: keyboard_controller
    run_name: collect_freeform_raven
    run_position_controller: position_controller_raven
    subrun_name: null